In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from __future__ import print_function

import numpy as np
import pandas as pd
import csv
import os

In [4]:
%ls -l data/*cnn*

-rw-r--r--  1 will  staff  250484 19 May 20:20 data/cnn-results.csv.bz2


In [9]:
cnn_results = pd.read_csv('data/cnn-results.csv.bz2')
y_test = cnn_results[['galaxy_type', 'z']]

In [12]:
predictions = np.zeros((5, len(y_test), 2))

for idx in range(1, 6):
    curr_predictions = predictions[idx - 1]
    curr_predictions[:,0] = cnn_results[f'pred_galaxy_type_val_{idx}']
    curr_predictions[:,1] = cnn_results[f'pred_photoz_{idx}']  

In [13]:
def get_y_results(predictions, y_test):
    predictions_avg = np.mean(predictions, axis=0)
    predictions_err = np.max(np.abs(predictions - predictions_avg), axis=0)
    predictions_std = np.std(predictions, axis=0, ddof=1)
    
    galaxy_classification_preds = predictions.transpose()[0]
    galaxy_classifications = np.count_nonzero(np.round(galaxy_classification_preds, 0), axis=1)
    y_results = y_test.copy().reset_index()
    y_results['galaxy_type_pred_raw'] = predictions_avg[:,0]
    y_results['galaxy_type_pred_err'] = predictions_err[:,0]
    y_results['galaxy_type_pred_std'] = predictions_std[:,0]
    y_results['galaxy_type_pred'] = np.round(y_results.galaxy_type_pred_raw,0)
    y_results['galaxy_type_pred'] = y_results['galaxy_type_pred'].astype(int, copy=False)
    y_results['elliptical_votes'] = galaxy_classifications
    y_results['galaxy_type_pred_votes'] = 0
    y_results.loc[y_results.elliptical_votes >= 3, 'galaxy_type_pred_votes'] = 1
    y_results['correct_votes'] = galaxy_classifications
    y_results.loc[y_results.galaxy_type == 0,'correct_votes'] = 5 - y_results['correct_votes']
    y_results['incorrect_votes'] = 5 - y_results['correct_votes']
    y_results['photoz'] = predictions_avg[:,1]
    y_results['photoz_err'] = (y_results['z'] - y_results['photoz'])/(1 + y_results['z'])
    y_results['photoz_err_abs'] = np.abs(y_results['photoz_err'])
    y_results['photoz_std'] = predictions_std[:,1]
    y_results['galaxy_err'] = np.abs(y_results.galaxy_type - y_results.galaxy_type_pred_raw)
    
    return y_results, galaxy_classifications

In [14]:
y_results, galaxy_classifications = get_y_results(predictions, y_test)